In [20]:
import pandas as pd
import numpy as np
import glob

In [ ]:
files = glob.glob(r"C:\Users\LucaG\Google Drive\Studium\Inhalt\Wirtschaftsinformatik\Thesis\Daten\Eingangstyp\*.csv")

df = pd.DataFrame()
for f in files:
    csv = pd.read_csv(f, sep=";")
    df = df.append(csv)

In [142]:
df = df.drop(columns=["6", "Erstellt am", "Aktiv"])

In [143]:
df.at[0,'ID']=1813819
df.at[1,'ID']=1813813
df.at[2,'ID']=1813814

In [145]:
df['Meldungseingang'] = df['Meldungseingang'].astype('str')

In [146]:
df['ID'].isna().sum()
df = df[df['ID'].notna()]

In [ ]:
df.columns = ["ticket_id", "input"]

In [149]:
df['ticket_id'] = df['ticket_id'].astype('int64')

In [ ]:
df = df.drop_duplicates("ticket_id")

In [153]:
data = pd.read_csv(r'C:\Users\LucaG\Google Drive\Studium\Inhalt\Wirtschaftsinformatik\Thesis\Daten\kpi_ticket_data_2019_2022.csv', sep=';')

In [129]:
data["kpi_field_name"] = np.where((data["kpi_operation"] == "create"), "create", data["kpi_field_name"])

In [ ]:
data = data.assign(sorting="")

In [131]:
data = data.drop(columns=["kpi_id", "kpi_timestamp_old", "kpi_field_value_old", "kpi_operation"])

In [133]:
data["sorting"] = np.where((data["kpi_field_name"] == "create"), 1 , data["sorting"])
data["sorting"] = np.where((data["kpi_field_name"] == "status"), 8 , data["sorting"])
data["sorting"] = np.where((data["kpi_field_name"] == "category"), 2 , data["sorting"])
data["sorting"] = np.where((data["kpi_field_name"] == "urgency"), 3 , data["sorting"])
data["sorting"] = np.where((data["kpi_field_name"] == "impact"), 4 , data["sorting"])
data["sorting"] = np.where((data["kpi_field_name"] == "priority"), 5 , data["sorting"])
data["sorting"] = np.where((data["kpi_field_name"] == "group"), 6 , data["sorting"])
data["sorting"] = np.where((data["kpi_field_name"] == "resolution_code"), 7 , data["sorting"])

In [ ]:
data = data.assign(typ="")

In [ ]:
data["typ"] = np.where((data["kpi_field_value"].str.startswith ("Incident")), "Incident", data["typ"])
data["typ"] = np.where((data["kpi_field_value"].str.startswith ("ServiceRequest")), "Service", data["typ"])
data["typ"] = np.where((data["kpi_field_value"].str.startswith ("auto")), "auto-unspezifiziert", data["typ"])

In [ ]:
data["kpi_field_name"] = np.where((data["kpi_field_name"] == "category"), data["typ"], data["kpi_field_name"])

In [ ]:
data["kpi_field_name"] = np.where((data["kpi_field_name"] == "status"), data["kpi_field_value"] , data["kpi_field_name"])

In [ ]:
data["kpi_field_name"] = np.where((data["kpi_field_name"] == "resolution_code"), data["kpi_field_value"] , data["kpi_field_name"])

In [ ]:
result = pd.merge(data, df, on="ticket_id", how="left")

In [157]:
result.drop_duplicates(ignore_index=True)
print(result)

         ticket_id            kpi_timestamp      kpi_field_name  \
0          1813813  2019-01-01 00:01:03.000              create   
1          1813813  2019-01-01 00:01:03.000      In Bearbeitung   
2          1813813  2019-01-01 00:01:03.000  vollständig gelöst   
3          1813813  2019-01-01 00:01:03.000               group   
4          1813813  2019-01-01 00:01:03.000            Incident   
...            ...                      ...                 ...   
4940393    3017561  2022-06-01 12:00:55.000            Incident   
4940394    3017561  2022-06-01 12:00:55.000            priority   
4940395    3017561  2022-06-01 12:00:55.000             urgency   
4940396    3017561  2022-06-01 12:00:55.000              impact   
4940397    3017561  2022-06-01 12:02:30.000               group   

                                           kpi_field_value sorting  \
0                                                      NaN       1   
1                                           In Bearbeit

In [165]:
result.groupby(['kpi_field_name']).describe()

ticket_id                               \
                                  count          mean            std   
kpi_field_name                                                         
Analyse                          4052.0  2.523048e+06  294581.761427   
Gelöst                         349806.0  2.488609e+06  294176.228718   
Geschlossen                    441917.0  2.354459e+06  339335.047430   
In Bearbeitung                 454509.0  2.398216e+06  335823.852880   
Incident                       172381.0  2.373424e+06  338433.548246   
Kunde wünscht schließen          8167.0  2.494565e+06  278461.701050   
Lösung in Kundenverantwortung   13674.0  2.395324e+06  354506.567256   
Lösung nicht notwendig          47124.0  2.374730e+06  320734.462917   
Obsolet durch Kunde              6688.0  2.485178e+06  294052.434817   
Offen                           49823.0  2.509469e+06  280228.582168   
Vorlage                            51.0  2.351190e+06  370960.179265   
Wartet - auf Angebot              850.0  2.589660e+06  228929.199910   
Wartet - auf Kunde              59447.0  2.430988e+06  334710.856338   
Wartet - auf Lieferung extern   21893.0  2.357565e+06  326521.907958   
Wartet - auf Release            16311.0  2.432198e+06  329649.985902   
Wartet - mit Kunde abgestimmt   26366.0  2.344321e+06  325793.405295   
Workaround (Übergangslösung)        4.0  1.842034e+06     641.378463   
auto-unspezifiziert             73710.0  2.275117e+06  341474.996262   
create                         431467.0  2.384429e+06  337479.486918   
group                          729882.0  2.382420e+06  342280.456692   
impact                         444478.0  2.373720e+06  341563.578273   
nicht gelöst                    10567.0  2.481796e+06  301460.942380   
nicht nachvollziehbar            4678.0  2.323341e+06  330075.476091   
nicht zuständig                 10796.0  2.536045e+06  251935.429776   
priority                       446472.0  2.373788e+06  341413.638619   
severity                            1.0  2.207687e+06            NaN   
teilweise gelöst - Workaround    8911.0  2.390755e+06  321266.691238   
template_name                      58.0  2.358098e+06  372205.493301   
urgency                        445865.0  2.373952e+06  341426.918664   
vollständig gelöst             350469.0  2.363883e+06  342008.675866   

                                                                             \
                                     min         25%        50%         75%   
kpi_field_name                                                                
Analyse                        1815041.0  2303747.50  2530293.5  2735538.75   
Gelöst                         1813816.0  2297442.25  2514290.5  2684659.75   
Geschlossen                    1813813.0  2017829.00  2391188.0  2636888.00   
In Bearbeitung                 1813813.0  2088265.00  2446537.0  2655794.00   
Incident                       1813813.0  2039139.00  2423205.0  2644383.00   
Kunde wünscht schließen        1814275.0  2336714.50  2495761.0  2680652.50   
Lösung in Kundenverantwortung  1813878.0  2037913.75  2448404.0  2675345.50   
Lösung nicht notwendig         1813817.0  2041056.75  2462052.5  2602486.00   
Obsolet durch Kunde            1814275.0  2281640.50  2503168.5  2682442.25   
Offen                          1813831.0  2279543.50  2510720.0  2692174.50   
Vorlage                        1840895.0  1999526.00  2351512.0  2664458.50   
Wartet - auf Angebot           1933272.0  2458740.00  2595041.0  2735160.50   
Wartet - auf Kunde             1813923.0  2192284.50  2486570.0  2678964.00   
Wartet - auf Lieferung extern  1813828.0  2031743.00  2391453.0  2629416.00   
Wartet - auf Release           1814179.0  2205819.00  2467737.0  2675161.00   
Wartet - mit Kunde abgestimmt  1813830.0  2026181.50  2385630.0  2580553.25   
Workaround (Übergangslösung)   1841530.0  1841563.00  1841849.0  1842319.50   
auto-unspezifiziert            1813874.0  1950660.25  2229851.5  2539407.

In [164]:
result['kpi_field_name'].replace('', np.nan, inplace=True)
result["kpi_field_name"].dropna()

0                      create
1              In Bearbeitung
2          vollständig gelöst
3                       group
4                    Incident
                  ...        
4940393              Incident
4940394              priority
4940395               urgency
4940396                impact
4940397                 group
Name: kpi_field_name, Length: 4630417, dtype: object

In [152]:
result.to_csv(r'C:\Users\LucaG\Google Drive\Studium\Inhalt\Wirtschaftsinformatik\Thesis\Daten\result.csv', index=False )